In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 5.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv

from torch_geometric.datasets import TUDataset

In [ ]:
m = nn.Dropout(p=0)
input = torch.randn(4, 4)
output = m(input)
print(input)
#print(output)

tensor([[ 1.2184,  0.1812,  1.6613, -0.0923],
        [ 0.8088, -1.8346, -0.2515,  0.1343],
        [ 1.8699,  1.8720,  1.7355, -0.4422],
        [-0.0989,  0.2519,  0.3898,  1.1324]])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!


In [ ]:

import sys 
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class GNN_GraphSage_Layer(nn.Module):
    '''
        #    A single GraphSage Layer: Graph Sampling and Aggregate
    '''
    def __init__(self, input_dim, output_dim, Bias, normalize_embedding, dropout, aggregation):
        super(GNN_GraphSage_Layer, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.Bias = Bias
        self.dropout = dropout
        self.normalize_embedding = normalize_embedding
        self.aggregation = aggregation
        #self.add_self = add_self
        
        if self.aggregation == 'mean':
            self.learnable_weights = nn.Linear(self.input_dim*2, self.output_dim, bias=self.Bias)
        else:
            self.learnable_weights = nn.Linear(self.input_dim, self.output_dim, bias=self.Bias)

        self.normalize = F.normalize
    
    def forward(self, input_tensor, tilda_adjacency_matrix):

        if self.aggregation == 'mean':
            tilda_adjacency_matrix = tilda_adjacency_matrix / tilda_adjacency_matrix.sum(-1, keepdim=True)
        num_node_per_graph = tilda_adjacency_matrix.size(1)
        print("num_node_per_graph: ", num_node_per_graph)

        tilda_adjacency_matrix_neghborhood = torch.mm(tilda_adjacency_matrix, input_tensor) # Y = A~ * X

        neighborhood_aggregated = torch.cat((tilda_adjacency_matrix_neghborhood, input_tensor), 1)

        node_linear = self.learnable_weights(neighborhood_aggregated) # Y * W
        
        if self.normalize_embedding:
            node_linear = self.normalize(node_linear, p=2, dim=1)
        


        return node_linear


In [ ]:
graphsage = GNN_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')
graph = dataset[0]
tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())))
#print(adjacency_matrix)

graphsage_output = graphsage(graph.x, tilda_adjacency_matrix)
print(graphsage)

num_node_per_graph:  17
GNN_GraphSage_Layer(
  (learnable_weights): Linear(in_features=14, out_features=7, bias=True)
)


In [ ]:
graphsage = GNN_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')
graph = dataset[0]
tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())))
#print(adjacency_matrix)

graphsage_output = graphsage(graph.x, tilda_adjacency_matrix)

num_node_per_graph:  17


In [ ]:
import sys 
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class GNN_Batched_GraphSage_Layer(nn.Module):
    '''
        #    A single GraphSage Layer: Graph Sampling and Aggregate
    '''
    def __init__(self, input_dim, output_dim, Bias, normalize_embedding, dropout, aggregation):
        super(GNN_Batched_GraphSage_Layer, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.Bias = Bias
        self.dropout = dropout
        self.normalize_embedding = normalize_embedding
        self.aggregation = aggregation
        #self.add_self = add_self
        
        if self.aggregation == 'mean':
            self.learnable_weights = nn.Linear(self.input_dim*2, self.output_dim, bias=self.Bias)
        else:
            self.learnable_weights = nn.Linear(self.input_dim, self.output_dim, bias=self.Bias)

        self.normalize = F.normalize
    
    
    def forward(self, new_features, tilda_adjacency_matrix):
        #tilda_adjacency_matrix, new_features = self.computational_matricess(batched_graphs)
        new_features = new_features.to(torch.float32)

    
        if self.aggregation == 'mean':
            tilda_adjacency_matrix = tilda_adjacency_matrix / tilda_adjacency_matrix.sum(-2, keepdim=True)
        num_node_per_graph = tilda_adjacency_matrix.size(1)

        tilda_adjacency_matrix_neghborhood = torch.bmm(tilda_adjacency_matrix, new_features) # Y = A~ * X

        neighborhood_aggregated = torch.cat((tilda_adjacency_matrix_neghborhood, new_features), 2)

        node_linear = self.learnable_weights(neighborhood_aggregated) # Y * W
        
        if self.normalize_embedding:
            node_linear = self.normalize(node_linear, p=2, dim=1)
        


        return node_linear

In [ ]:
mat = torch.arange(18).view(2,3, -1)
print(mat)

print(torch.sum(mat, dim=-1))



tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]])
tensor([[ 3, 12, 21],
        [30, 39, 48]])


In [ ]:
t = torch.tensor([[1.,2.,3.]])
print("tensor: ", t)
t1 = F.normalize(t, p=2.0, dim = 0)
t2 = F.normalize(t, p=2.0, dim = 1)
print(t1)
print(t2)

tensor:  tensor([[1., 2., 3.]])
tensor([[1., 1., 1.]])
tensor([[0.2673, 0.5345, 0.8018]])


In [ ]:
from torch_geometric.loader import DataLoader
batched_dataset = DataLoader(dataset, batch_size=20, shuffle=False)
batched_graphsage_example = GNN_Batched_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')
print(batched_graphsage_example)

GNN_Batched_GraphSage_Layer(
  (learnable_weights): Linear(in_features=14, out_features=7, bias=True)
)


In [ ]:
for batched_graph in batched_dataset:
    #print(len(batched_graph))
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
    #print(batched_graph.num_graphs)
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    #print("tilda_adjacency_matrix: ", tilda_adjacency_matrix.size())
    softmaxed_output = batched_graphsage_example(batched_graph)
    print("softmaxed_output: ", softmaxed_output.size())
    break
print(softmaxed_output)

softmaxed_output:  torch.Size([20, 16, 7])
tensor([[[-0.2269, -0.2606, -0.2084,  ..., -0.2661,  0.2555, -0.1350],
         [-0.2269, -0.2606, -0.2084,  ..., -0.2661,  0.2555, -0.1350],
         [-0.2038, -0.2728, -0.2191,  ..., -0.2553,  0.2572, -0.1282],
         ...,
         [-0.1806, -0.2851, -0.2299,  ..., -0.2445,  0.2588, -0.1215],
         [ 0.5219, -0.1534, -0.5636,  ..., -0.1558,  0.2297, -0.8279],
         [ 0.0251,  0.2255, -0.3063,  ...,  0.0689,  0.1843, -0.1422]],

        [[ 0.0765,  0.0498, -0.5012,  ...,  0.0567,  0.1877,  0.0596],
         [-0.2249, -0.2889, -0.1493,  ..., -0.2998,  0.2661, -0.1027],
         [-0.2020, -0.3025, -0.1570,  ..., -0.2877,  0.2678, -0.0976],
         ...,
         [ 0.0391,  0.2199, -0.2385,  ...,  0.0745,  0.1937, -0.0859],
         [-0.2249, -0.2889, -0.1493,  ..., -0.2998,  0.2661, -0.1027],
         [-0.2249, -0.2889, -0.1493,  ..., -0.2998,  0.2661, -0.1027]],

        [[-0.1646, -0.2260, -0.1266,  ..., -0.2173,  0.2629, -0.1651],
  

In [ ]:
print(graphsage_output.size())
print(graphsage_output)

torch.Size([17, 7])
tensor([[ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.6193, -0.0558,  0.3279, -0.2110, -0.5358, -0.3831,  0.1656],
        [ 0.5391, -0.0586,  0.2

In [ ]:
sum=0
length_list = []
for i in range(len(dataset)):
    sum = sum + len(dataset[i].x)
    length_list.append(len(dataset[i].x))
    #print(len(dataset[i].x))
print("number of nodes in whole dataset: ", sum)
print(length_list)
print("max number of nodes in graphs: ", max(length_list))

number of nodes in whole dataset:  3371
[17, 13, 13, 19, 11, 28, 16, 20, 12, 17, 17, 20, 22, 13, 19, 22, 11, 17, 13, 18, 18, 17, 23, 27, 17, 13, 23, 17, 23, 23, 22, 24, 23, 13, 17, 14, 17, 15, 15, 13, 17, 13, 19, 17, 12, 23, 22, 17, 20, 16, 26, 26, 19, 19, 14, 17, 21, 25, 23, 19, 17, 11, 23, 20, 16, 16, 20, 23, 19, 14, 26, 16, 16, 23, 18, 10, 16, 16, 17, 19, 12, 25, 16, 11, 23, 23, 16, 12, 13, 23, 25, 19, 23, 19, 19, 25, 18, 13, 15, 16, 23, 26, 19, 23, 17, 20, 25, 19, 28, 24, 11, 15, 13, 16, 12, 10, 21, 23, 21, 13, 25, 21, 17, 11, 19, 20, 21, 15, 14, 11, 19, 11, 21, 22, 11, 20, 22, 13, 11, 16, 11, 20, 12, 11, 16, 16, 13, 15, 20, 12, 12, 22, 15, 12, 14, 12, 20, 20, 20, 14, 26, 24, 22, 22, 19, 23, 22, 11, 13, 17, 24, 12, 21, 22, 22, 12, 20, 13, 22, 28, 11, 14, 22, 22, 13, 12, 21, 16]
max number of nodes in graphs:  28


In [ ]:

class DiffPool_Embedding_Layer(nn.Module):
    '''
   #     Z, new features size
    '''
    def __init__(self, input_dim_size, new_feat_dim_size, Bias, normalize_embedding, dropout, aggregation):
        super(DiffPool_Embedding_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        self.embedding_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        #self.embedding_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu


    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
        z_l_init = self.embedding_layer(input_tensor, tilda_adjacency_matrix)
        z_l_init = self.act_fun(z_l_init)

        return z_l_init


In [ ]:
import sys 
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class Batched_DiffPool_Embedding_Layer(nn.Module):
    '''
   #     Z, new features size
    '''
    def __init__(self, input_dim_size, new_feat_dim_size, Bias, normalize_embedding, dropout, aggregation):
        super(Batched_DiffPool_Embedding_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        #self.embedding_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.embedding_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu


    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
        z_l_init = self.embedding_layer(input_tensor, tilda_adjacency_matrix)
        z_l_init = self.act_fun(z_l_init)

        return z_l_init


In [ ]:
#import binascii

class DiffPool_Assignment_Layer(nn.Module):
    '''
    #    S, new clusters, new number of nodes
    '''
    def __init__(self, input_dim_size, new_num_nodes, Bias, normalize_embedding, dropout, aggregation):
        super(DiffPool_Assignment_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        self.assinment_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        #self.assinment_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu
    
    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
        s_l_init = self.assinment_layer(input_tensor, tilda_adjacency_matrix)
        s_l_init = self.act_fun(s_l_init)

        s_l = F.softmax(s_l_init, dim=-1)
        
        return s_l


In [ ]:
import sys 
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util

class Batched_DiffPool_Assignment_Layer(nn.Module):
    '''
    #    S, new clusters, new number of nodes
    '''
    def __init__(self, input_dim_size, new_num_nodes, Bias, normalize_embedding, dropout, aggregation):
        super(Batched_DiffPool_Assignment_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        #self.assinment_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.assinment_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu
    
    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
        s_l_init = self.assinment_layer(input_tensor, tilda_adjacency_matrix)
        s_l_init = self.act_fun(s_l_init)

        s_l = F.softmax(s_l_init, dim=-1)
        
        return s_l


In [ ]:

class DiffPool_Layer(nn.Module):
    def __init__(self, input_dim_size, new_feat_dim_size, new_num_nodes, Bias, normalize_embedding, dropout, aggregation):
        super(DiffPool_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation

        self.new_embed = DiffPool_Embedding_Layer(input_dim_size=input_dim_size, new_feat_dim_size=new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.new_assign = DiffPool_Assignment_Layer(input_dim_size=self.input_dim_size, new_num_nodes=self.new_num_nodes, Bias=self.Bias , normalize_embedding=self.normalize_embedding , dropout=self.dropout, aggregation=self.aggregation)

        
    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 

        z_l = self.new_embed(input_tensor, tilda_adjacency_matrix)
        s_l = self.new_assign(input_tensor, tilda_adjacency_matrix)

        new_X = torch.mm(s_l.transpose(-1, -2), z_l)
        new_adjacency = (s_l.transpose(-1, -2)).mm(tilda_adjacency_matrix).mm(s_l)

        return new_X, new_adjacency


In [ ]:
import sys 
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class Batched_DiffPool_Layer(nn.Module):
    def __init__(self, input_dim_size, new_feat_dim_size, new_num_nodes, Bias, normalize_embedding, dropout, aggregation):
        super(Batched_DiffPool_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation

        self.new_embed = Batched_DiffPool_Embedding_Layer(input_dim_size=input_dim_size, new_feat_dim_size=new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.new_assign = Batched_DiffPool_Assignment_Layer(input_dim_size=self.input_dim_size, new_num_nodes=self.new_num_nodes, Bias=self.Bias , normalize_embedding=self.normalize_embedding , dropout=self.dropout, aggregation=self.aggregation)  


    def forward(self, new_features, new_adjacecny):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 
        #new_adjacecny, new_features = self.computational_matricess(batched_graphs)

        z_l = self.new_embed(new_features, new_adjacecny)
        s_l = self.new_assign(new_features, new_adjacecny)

        print("z_l: ", z_l.size())
        print("s_l: ", s_l.size())
        new_X = torch.bmm(s_l.transpose(-1, -2), z_l)
        new_adjacency = (s_l.transpose(-1, -2)).bmm(new_adjacecny).bmm(s_l)

        return new_X, new_adjacency


In [ ]:
batched_diffpool_layer_example = Batched_DiffPool_Layer(input_dim_size=7, new_feat_dim_size=8, new_num_nodes=13, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')

In [ ]:
from torch_geometric.loader import DataLoader
batched_dataset = DataLoader(dataset, batch_size=30, shuffle=False)
for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y 

    new_X, new_adjacency = batched_diffpool_layer_example(batched_graph)
    print("softmaxed_output: ", new_X.size())
    print("softmaxed_output: ", new_adjacency.size())
    break

z_l:  torch.Size([30, 17, 8])
s_l:  torch.Size([30, 17, 13])
softmaxed_output:  torch.Size([30, 13, 8])
softmaxed_output:  torch.Size([30, 13, 13])


In [ ]:
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys 
from torch_geometric.datasets import TUDataset
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import Batched_GraphSage_Layer as batched_graphsage_layer
import Batched_DIFFPOOL_Assignment as batched_diffpool_assignment
import Batched_DIFFPOOL_Embedding as batched_diffpool_embedding
import Batched_DIFFPOOL_Layer as batched_diffpool_layer



class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DIFFPOOL_Model(nn.Module):
    '''
        DIFFPOOL Mode
    '''
    def __init__(self, diffpool_layers_dim, diffpool_layers_new_num_nodes, Weight_Initializer, Bias, num_classes, dropout_rate, normalize_embedding, aggregation, act_fun):
        
        super(DIFFPOOL_Model, self).__init__()
        self.diffpool_layers_dim = diffpool_layers_dim
        self.diffpool_layers_new_num_nodes = diffpool_layers_new_num_nodes
        self.dropout_rate = dropout_rate
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.aggregation = aggregation
        self.num_classes = num_classes

                                
        self.diffpool_layer_1 = batched_diffpool_layer.Batched_DiffPool_Layer(input_dim_size=self.diffpool_layers_dim[0][0], new_feat_dim_size=self.diffpool_layers_dim[0][1], new_num_nodes=self.diffpool_layers_new_num_nodes[0], Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout_rate, aggregation=self.aggregation)

        self.graph_sage_1 = batched_graphsage_layer.GNN_Batched_GraphSage_Layer(input_dim=self.diffpool_layers_dim[0][1], output_dim=self.diffpool_layers_dim[1][0], Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout_rate, aggregation=self.aggregation)

        self.diffpool_layer_2 = batched_diffpool_layer.Batched_DiffPool_Layer(input_dim_size=self.diffpool_layers_dim[1][0], new_feat_dim_size=self.diffpool_layers_dim[1][1], new_num_nodes=self.diffpool_layers_new_num_nodes[1], Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout_rate, aggregation=self.aggregation)

        self.graph_sage_2 = batched_graphsage_layer.GNN_Batched_GraphSage_Layer(input_dim=self.diffpool_layers_dim[1][1], output_dim=self.diffpool_layers_dim[1][1], Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout_rate, aggregation=self.aggregation)

        self.lin1 = torch.nn.Linear(in_features=self.diffpool_layers_dim[1][1], out_features=self.diffpool_layers_dim[1][1])
        self.lin2 = torch.nn.Linear(in_features=self.diffpool_layers_dim[1][1], out_features=self.diffpool_layers_dim[1][1])
        self.lin3 = torch.nn.Linear(in_features=self.diffpool_layers_dim[1][1], out_features=self.num_classes)
        
        if act_fun == 'ReLu':
            self.act_fun = F.relu
            print('ReLu is Selected.')
        elif act_fun == 'eLu':
            self.act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif act_fun == 'tanh':
            self.act_fun = torch.tanh
            print('tanh is Selected.')
        self.act_fun_softmax = F.softmax       
        

        mean = 0
        std = 0.1
        self.initialize_weights(Weight_Initializer, Bias, mean, std)
    

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                            torch.nn.init.xavier_normal_(layer.learnable_weights.weight)
                            if Bias:
                                layer.learnable_weights.bias.data.zero_()
                        else:
                            pass
                elif isinstance(layers, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                    torch.nn.init.xavier_normal_(layers.learnable_weights.weight)
                    if Bias:
                        layers.learnable_weights.bias.data.zero_()
                elif isinstance(layers, batched_diffpool_layer.Batched_DiffPool_Layer):
                    torch.nn.init.xavier_normal_(layers.new_assign.assinment_layer.learnable_weights.weight)
                    torch.nn.init.xavier_normal_(layers.new_embed.embedding_layer.learnable_weights.weight)
                    if Bias:
                        torch.nn.init.zeros_(layers.new_assign.assinment_layer.learnable_weights.bias)
                        torch.nn.init.zeros_(layers.new_embed.embedding_layer.learnable_weights.bias)
                elif isinstance(layers, torch.nn.Linear):
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        torch.nn.init.zeros_(layers.bias)
                    

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                            torch.nn.init.kaiming_normal_(layer.learnable_weights.weight)
                            if Bias:
                                layer.learnable_weights.bias.data.zero_()
                        else:
                            pass
                elif isinstance(layers, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                    torch.nn.init.kaiming_normal_(layers.learnable_weights.weight)
                    if Bias:
                        layers.learnable_weights.bias.data.zero_()
                elif isinstance(layers, batched_diffpool_layer.Batched_DiffPool_Layer):
                    torch.nn.init.kaiming_normal_(layers.new_assign.assinment_layer.learnable_weights.weight)
                    torch.nn.init.kaiming_normal_(layers.new_embed.embedding_layer.learnable_weights.weight)
                    if Bias:
                        torch.nn.init.zeros_(layers.new_assign.assinment_layer.learnable_weights.bias)
                        torch.nn.init.zeros_(layers.new_embed.embedding_layer.learnable_weights.bias)
                elif isinstance(layers, torch.nn.Linear):
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        torch.nn.init.zeros_(layers.bias)
                    
                            
        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                            torch.nn.init.normal_(layer.learnable_weights.weight.data, mean, std)
                            if Bias:
                                layer.learnable_weights.bias.data.zero_()
                        else:
                            pass
                elif isinstance(layers, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                    torch.nn.init.normal_(layers.learnable_weights.weight, mean=mean, std=std)
                    if Bias:
                        layers.learnable_weights.bias.data.zero_()
                elif isinstance(layers, batched_diffpool_layer.Batched_DiffPool_Layer):
                    torch.nn.init.normal_(layers.new_assign.assinment_layer.learnable_weights.weight, mean=mean, std=std)
                    torch.nn.init.normal_(layers.new_embed.embedding_layer.learnable_weights.weight, mean=mean, std=std)
                    if Bias:
                        torch.nn.init.zeros_(layers.new_assign.assinment_layer.learnable_weights.bias)
                        torch.nn.init.zeros_(layers.new_embed.embedding_layer.learnable_weights.bias)
                elif isinstance(layers, torch.nn.Linear):
                    torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                    if Bias:
                        torch.nn.init.zeros_(layers.bias)
                    

    def computational_matricess(self, batched_graphs):
        joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
        batch_size = batched_graphs.num_graphs

        #print("whole_graphs_adjacency.size()[0]: ", whole_graphs_adjacency.size()[0])
        new_number_of_nodes = int(joint_tilda_adjacency_matrix.size()[0] / batch_size)
        #print(batch_size)
        adjacency_list = []
        feature_list = []
        for i in range(batch_size):
            start = i * new_number_of_nodes
            end = (i + 1) * new_number_of_nodes
            adjacency_list.append(joint_tilda_adjacency_matrix[start:end, start:end])
            feature_list.append(batched_graphs.x[start:end, :])
        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))
        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)
        new_adjacecny = new_adjacecny.view(batch_size, new_number_of_nodes,new_number_of_nodes)

        return new_adjacecny, new_features  

    
    def forward(self, batched_graphs):
        new_adjacecny, new_features = self.computational_matricess(batched_graphs)
        
        #if batch is not None:
        #    max_dim = 1
        #else:
        #    max_dim = 0

        new_features = new_features.to(torch.float32)

        new_X, new_adjacency_1 = self.diffpool_layer_1(new_features, new_adjacecny)
        graph_sage1_output = self.graph_sage_1(new_X, new_adjacency_1)

        new_X_2, new_adjacency_2 = self.diffpool_layer_2(graph_sage1_output, new_adjacency_1)
        graph_sage2_output = self.graph_sage_2(new_X_2, new_adjacency_2)

        #graph_sage2_output = graph_sage2_output.view((len(graph), self.diffpool_layers_new_num_nodes[1], self.diffpool_layers_dim[1][1]))
        graph_sage2_output, q = torch.max(graph_sage2_output, dim=1, keepdim=True)

        linear1_output = self.lin1(graph_sage2_output)
        linear1_output = self.act_fun(linear1_output)

        linear2_output = self.lin2(linear1_output)
        linear2_output = self.act_fun(linear2_output)

        linear3_output = self.lin3(linear2_output)
        linear3_output = self.act_fun_softmax(linear3_output, dim=2)

        
        return linear3_output




dataset = TUDataset(root='data/TUDataset', name='MUTAG')
sum=0
length_list = []
for i in range(len(dataset)):
    sum = sum + len(dataset[i].x)
    length_list.append(len(dataset[i].x))

#print("max number of nodes in graphs: ", max(length_list))
batch_size = 32
new_num_nodes = 10#int(length_list[0] / batch_size)
print("new_num_nodes: ", new_num_nodes)

node_feat_size = len(dataset[0].x[0])

hid_dim = 7

diffpool_model_example = DIFFPOOL_Model(diffpool_layers_dim=[[node_feat_size, hid_dim], [hid_dim, node_feat_size]], diffpool_layers_new_num_nodes=[new_num_nodes, new_num_nodes], 
                                        Weight_Initializer=1, Bias=True, num_classes=2, dropout_rate=0, normalize_embedding=True, aggregation='mean', 
                                        act_fun='ReLu')

batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    linear3_output = diffpool_model_example(batched_graph)
    print("Final Output: ", linear3_output)


new_num_nodes:  10
ReLu is Selected.
z_l:  torch.Size([32, 18, 7])
s_l:  torch.Size([32, 18, 10])
z_l:  torch.Size([32, 10, 7])
s_l:  torch.Size([32, 10, 1])
Final Output:  tensor([[[0.4560, 0.5440]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.3093, 0.6907]],

        [[0.4560, 0.5440]],

        [[0.1369, 0.8631]],

        [[0.3093, 0.6907]],

        [[0.4560, 0.5440]],

        [[0.4560, 0.5440]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.4560, 0.5440]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.1369, 0.8631]],

        [[0.3342, 0.6658]],

        [[0.4560, 0.5440]],

        [[0.4560, 0.5440]],

        [[0.3761, 0.6239]],

        [[0.4560, 0.5440]],

        [[0.1369, 0.8631]],

        [[0.4560, 0.5440]],

        [[0.3093, 0.6907]],

        [[0.3093, 0.6907]],

        [[0.334

In [ ]:
for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    #print("tilda_adjacency_matrix: ", tilda_adjacency_matrix.size())
    gnn_sage1_output = diffpool_model_example(batched_graph)
    #print("gnn_sage1_output: ", gnn_sage1_output.size())
    print(gnn_sage1_output)
    #break

In [ ]:
output_diffpool = diffpool_model_example(dataset[2])
print(output_diffpool)

tensor([[[0.5012, 0.4988],
         [0.5012, 0.4988],
         [0.5012, 0.4988],
         [0.5012, 0.4988],
         [0.5012, 0.4988],
         [0.5012, 0.4988],
         [0.5012, 0.4988]]], grad_fn=<SoftmaxBackward0>)


In [ ]:
def batching_adjacency(whole_graphs_adjacency, batch_feat, batch_size):
    """
        transform a batched graph to batched adjacency tensor and node feature tensor
    """
    #print("whole_graphs_adjacency.size()[0]: ", whole_graphs_adjacency.size()[0])
    new_number_of_nodes = int(whole_graphs_adjacency.size()[0] / batch_size)
    #print(batch_size)
    adjacency_list = []
    feature_list = []
    for i in range(batch_size):
        start = i * new_number_of_nodes
        end = (i + 1) * new_number_of_nodes
        adjacency_list.append(whole_graphs_adjacency[start:end, start:end])
        feature_list.append(batch_feat[start:end, :])
    adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
    feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))
    new_adjacecny = torch.cat(adjacency_list, dim=0)
    new_features = torch.cat(feature_list, dim=0)

    return new_features, new_adjacecny

In [ ]:
batch_size = 32
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    feat,adj = batching_adjacency(tilda_adjacency_matrix, x, batch_size)
    break
print(adj.size())
print(feat.size())

whole_graphs_adjacency.size()[0]:  585
32
torch.Size([32, 18, 18])
torch.Size([32, 18, 7])


In [ ]:
num_graphs=20
n = 7
adj=torch.randint(0, 1, (num_graphs, n, n))
offset, row, col = (batch.adj > 0).nonzero().t()
edge_weight = adj[offset, row, col]
row += offset * n
col += offset * n
edge_index = torch.stack([row, col], dim=0)
x = x.view(num_graphs * n, num_feats)
batch = torch.arange(0, num_graphs).view(-1, 1).repeat(1, n).view(-1)

AttributeError: ignored

In [ ]:
        #tilda_degree_vector = torch.sum(tilda_adjacency_matrix, dim=1)

        #k = tilda_degree_vector.size(0)
        #tilda_degree_matrix = torch.zeros(k, k)
        #tilda_degree_matrix.as_strided([k], [k + 1]).copy_(tilda_degree_vector)
        #tilda_degree_matrix = tilda_degree_matrix.to(torch.float32)

        #ones_matrix = torch.ones_like(tilda_degree_matrix)

        #reciprocal_tilda_degree_matrix = ones_matrix.div(tilda_degree_matrix)
        #reciprocal_tilda_degree_matrix = torch.nan_to_num(reciprocal_tilda_degree_matrix, nan=0, posinf=0, neginf=0)

        #return tilda_adjacency_matrix, reciprocal_tilda_degree_matrix
        #tilda_degree_matrix = torch.nan_to_num(tilda_degree_matrix, nan=0, posinf=0, neginf=0)